In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as met
import sklearn.model_selection as model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import numpy as np

In [ ]:
bank_df = pd.read_csv('/kaggle/input/bank-loan/Bank_Personal_Loan_Modelling.csv',error_bad_lines=False)

<font color="blue"> <b>EDA</b> - Exploratory Data Analytics</font>

In [ ]:
bank_df.columns

In [ ]:
bank_df.shape

In [ ]:
bank_df.dtypes

In [ ]:
bank_df.info()

In [ ]:
bank_df.apply(lambda val: sum(val.isnull()))

<font color="green">Hence ensured no missing values</font>

In [ ]:
bank_df.describe().T

<font color="#07bdf5"> Observation on given data
using info()</font>
<ol>
<li>All attributes contains numeric data, may be few are categorical</li>
<li>All attributes are non-null, hence no missing values</li>
<li>There are 13 attributes</li>
</ol>
<font color="#07bdf5">Using describe()</font>
<ol>
<li>Categorical attributes are : Age, Zip,
    <ul>
        <li>Binary Categorical(5)</li>
            <ol> 
                <li><font color="#11aaa1">Personal Loan: Did this customer accept the personal loan offered in the last campaign?</font><font color="red"> &lt;&lt;&lt;--this is the dependent / target attribute</font></li>
                <li>Securities Account: Does the customer have a securities account with the bank?</li>
                <li>CD AccountDoes the customer have a certificate of deposit (CD) account with the bank?</li>
                <li>Online: Does the customer use internet banking facilities?</li>
                <li>CreditCard: Does the customer use a credit card issued by Bank?</li>
            </ol>
        <li>Ordinal Categorical (2)</li>
            <ol>
                <li>Education: Education Level. 1: Undergrad; 2: Graduate;3: Advanced/Professional</li>
                <li>Family: Family size of the customer</li>
            </ol>
    </ul>                
</li>
<li>Continuous attributes are (5): Experience, Income, CCAvg, Mortgages</li>
<li> Nominal attributees are : Zip and ID (index column need not be consider, as it is not adding any value to the model) </li>
    </ol>
    <font color="#07bdf5">Other observations</font>
    <ul>
    <li>Experieice min values is -3, must be a wrong data entry</li>
    <li>Experience attaribute is normally distributed, where mean id 20, and First quantile(25%) 10, second quantile(50%) 20, third quantile(75%) is 30 and max is 43. Mean and Median is 20, Customer's experience ranges from no experience to max age of 43</li>
    <li>Similarly analyzing Income Min 8, max 224, 50% is 64 75% is 98 - seems to be right skewed / positevely skewed</li>
    <li>CCAve average spending min 0 max 10, mean 1.9, mean 1.9, 2nd quantile 1.5, 3rd quantile 2.5 majority data between 3rd and max, i.e., 2.5 to 10 - right skewed / positevely skewed </li>
    </ul> 
</font>

In [ ]:
sns.pairplot(data=bank_df)

In [ ]:
bank_df.corr()

In [ ]:
import pylab as pl
pl.figure(figsize = (16,10))

sns.heatmap(bank_df.corr(), annot=True, fmt='0.2f')

<font color="#07bdf5">Other observations</font>
<ul>
    <li>CCAvg and Personal loan moderately corralated</li>
    <li>Experience and Age are hily coralated</li>

In [ ]:
bank_df.head(10)

In [ ]:
bank_df[["Experience","Income","CCAvg", "Mortgage"]].apply(lambda x: pd.Series([(x < 0).sum(), (x > 0).sum(), (x==0).sum()]))
# 0 - count of negative values
# 1 - count of positive values
# 1 - count of zero values

In [ ]:
sns.boxplot(x=bank_df.Family,y=bank_df.Income,hue=bank_df["Personal Loan"])

<p><font color="#07bdf5">More observations</font>
    Attribute <b>Experience </b>has <font color="red">52</font> negative values, probably data entry issue</p>
<p>
  <font color="blue">  <ul>
       <li>Take away for Business: Families with income more than USD100,000 are availed the personal loan during previous campaign, Families Income below $100,000 are less likely to avail personal loan </li>
        </ul></font>

In [ ]:
bank_df["Personal Loan"].unique()

In [ ]:
bank_df.groupby(["Personal Loan"]).count()

<font color="blue">ID column not adding value to the data, hence droping it</font>

In [ ]:
bank_df = bank_df.drop(["ID"],axis=1)

<h2><font color="blue"> Creation of Model</font></h2>
<b>Itreation 1</b>

In [ ]:
#splitting training and testing data
X_train, X_test = train_test_split(bank_df , test_size=0.3, random_state=1)
y_train = X_train.pop("Personal Loan")
y_test = X_test.pop("Personal Loan")
modelcrorecard={} #defining a dictionary to comparing different model

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

<font color="blue"><b> Logistic Regression</b> <font color="maroon">using StatsModels</font></font>

In [ ]:
def calculate_accuracy(confusion_matrix):
    accuracy = (confusion_matrix[0][0]+confusion_matrix[1][1])/(confusion_matrix[0][0]+confusion_matrix[0][1]+confusion_matrix[1][0]+confusion_matrix[1][1])
    return accuracy

In [ ]:
def get_predictions( X_testLR1, logitmodel ):
    y_pred_df = pd.DataFrame( { 'actual': y_test,
                               "predicted_prob": logitmodel.predict( sm.add_constant( X_testLR1 ) ) } )
    return y_pred_df

In [ ]:
import statsmodels.api as sm

def do_statmodelLR(X_trainLR1,X_testLR1,y_trainLR1,y_testLR1):
    logit = sm.Logit( y_trainLR1, sm.add_constant( X_trainLR1 ) )
    logitmodel = logit.fit()
    y_pred_bank_df = get_predictions(X_testLR1, logitmodel )
    y_pred_bank_df["original"] = np.array(y_test)
    y_pred_bank_df['predicted'] = y_pred_bank_df.predicted_prob.map( lambda x: 1 if x > 0.6 else 0)
    return y_pred_bank_df,logitmodel

In [ ]:
pred_y_df,lgmetric = do_statmodelLR(X_train,X_test,y_train,y_test)
con_matrix = confusion_matrix(pred_y_df.original,pred_y_df.predicted)

In [ ]:
from sklearn import metrics
def draw_conmmat( actual, predicted, modelname): #creating this as function as I am planning to reuse
    cm = metrics.confusion_matrix( actual, predicted, [1,0] )
    sns.heatmap(cm, annot=True,  fmt='.2f', xticklabels = ["Buy Personal Loan", "No Buy Personal Loan"] , yticklabels = ["Buy Personal Loan", "No Buy Personal Loan"] )
    plt.ylabel('True label')
    plt.xlabel('Predicted label - '+str(modelname))
    plt.show()

In [ ]:
draw_conmmat( pred_y_df.original, pred_y_df.predicted, "Logistic Regression using StatsModels")

In [ ]:
acc11 = calculate_accuracy(con_matrix)
modelcrorecard["LRstatmodel"]=acc11
print(acc11)

In [ ]:
bank_df.groupby(["Personal Loan"]).count()

<font color="blue">Original data contains more number of 0's meaning - more number of customer not accepted the personal loan offered in the last campaign, so it will predit - not availed correctly</font>

In [ ]:
#Total accuracy  (TP+TN)/(TP+FP+TN+FN)
accuracy = (80+1343)/(80+69+1343+8)
print("Accuracy :"+ str(accuracy))

<font color="blue"><b>Doing LogisticRegression with <font color="color">scikit sklearn,</font> instead of statsmodels.api</b><font>

In [ ]:
from sklearn.linear_model import LogisticRegression

def do_LR(X_trainLR,y_trainLR,X_testLR,y_testLR):
    clf = LogisticRegression(fit_intercept = False, C = 1e9) 
    #scikit-learn's logistic regression performs regularization by default. To negate setting high value for C
    #did this and verified to verify whether scikit learn can do same as statsmodel
    clf.fit(X_trainLR,y_trainLR) #Model
    y_pred = clf.predict(X_testLR) #prediction
    scikit_score = clf.score(X_testLR,y_testLR)
    return scikit_score,y_pred

In [ ]:
LRScore, LRpred_labels = do_LR(X_train,y_train,X_test,y_test)
print("Score scikit learn Logistic Regression : "+str(LRScore))
modelcrorecard["LRscikitlearn"]=LRScore

In [ ]:
confusion_matrix(y_test,LRpred_labels)

In [ ]:
draw_conmmat(y_test,LRpred_labels, "Logistic Regression using scikit learn")

<font color="blue"><b>KNN</b></font>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def do_KNN(X_train,y_train,y_test, K_value): #defining this as function as I have to reuse
    NNH = KNeighborsClassifier(n_neighbors= K_value , weights = 'uniform', metric='euclidean' )
    NNH.fit(X_train, y_train)
    # For every test data point, predict it's label based on 5 nearest neighbours in this model. The majority class will 
    # be assigned to the test data point
    KNNpredict = NNH.predict(X_test)
    score = NNH.score(X_test, y_test)
    return score,KNNpredict

In [ ]:
for i in range(1,21): #to find optimum K
    KNNscore,KNNpredict = do_KNN(X_train,y_train,y_test,i)# applying K values from 1 to 20
    print("When K value is "+str(i)+" score = "+str(KNNscore))

<font color="blue"> When K value is 3 the score is 90% which is higher than other K values, hence the best K value for this model is 3</font>

In [ ]:
# applying K value as 3
optimum_kvalue = 3
KNNscore,KNNpredict = do_KNN(X_train,y_train,y_test,optimum_kvalue)
print("When K value is "+str(optimum_kvalue)+" score = "+str(KNNscore))
print("Confusion matrix for KNN is")
conf_matrix= confusion_matrix(y_test,KNNpredict)
modelcrorecard["KNN"] = KNNscore

In [ ]:
draw_conmmat(y_test,KNNpredict,"KNN")

In [ ]:
calculate_accuracy(conf_matrix)

In [ ]:
acc_knn=(1333+21)/(1333+21+128+18) #Just to verify through manual calculation
print("KNN Accuracy="+str(acc_knn))

<font color="blue"><b>Naïve Bayes</b></font>

In [ ]:
from sklearn.naive_bayes import GaussianNB 

def do_naive_bayes(X_trainNB,y_trainNB,X_testNB,y_testNB):
    
    naive_model = GaussianNB()
    naive_model.fit(X_trainNB, y_trainNB)

    NBpredict = naive_model.predict(X_testNB)
    NBscore = naive_model.score(X_testNB,y_testNB)
    return NBscore,NBpredict

In [ ]:
NBscore, NBpredicted_labels = do_naive_bayes(X_train,y_train,X_test,y_test)
print("Score in Naive Bayes : ",NBscore)
modelcrorecard["Naive Bayes"]=NBscore

In [ ]:
print("Confusion matrix for Naive Bayes : ")
print(y_test.shape,"\n\n",NBpredicted_labels.shape)
#con_matrix = confusion_matrix(y_test.tolist(), NBpredicted_labels.tolist())
draw_conmmat(y_test,NBpredicted_labels,"Naive Bayes")

<font color="blue"><b>Iteration 2</b></font>
<h2>Removing 52 negative values in Experience column</h2>

In [ ]:
bank_df_corrected = bank_df.copy() #Take a copy of bank_df keeping the original data as is
bank_df_corrected[bank_df_corrected[["Experience"]] < 0] = -1 # filling negative values as NaN

In [ ]:
# After replacing negative values with NaN, if we check no non negative values
bank_df_corrected[["Experience"]].apply(lambda x: pd.Series([(x < 0).sum(), (x > 0).sum(), (x == 0).sum()]))
# 0 - count of negative values
# 1 - count of positive values
# 1 - count of zero values

In [ ]:
#find mean of Experience with NaN
bank_df_corrected["Experience"].mean()

In [ ]:
#Fill negative values with mean
exp_array = bank_df_corrected[bank_df_corrected["Experience"]!= -1]["Experience"]
bank_df_corrected["Experience"]=bank_df_corrected["Experience"].replace(-1,exp_array.mean())
print(exp_array.mean())

<font color="blue">Earlier the mean was 20 now it has become 20.33 let us replace NaN with mean experieice and verify the mean again</font>

In [ ]:
# After replacing negative values with NaN, if we check no non negative values
bank_df_corrected[["Experience"]].apply(lambda x: pd.Series([(x < 0).sum(), (x > 0).sum(), (x == 0).sum()]))
# 0 - count of negative values
# 1 - count of positive values
# 1 - count of zero values

In [ ]:
for key, val in modelcrorecard.items():
    print(key,"",val)

<table>
<tr>
    <td><b>Model</b></td>
    <td><b>Score</b></td>
</tr>
<tr>
<td>Logistic Regression-statmodel</td>
<td>0.9453333333333334</td>
</tr>
<tr>
<td>Logistic Regression-scikit learn</td>
<td>0.9073333333333333</td>
</tr>
<tr>
<td>KNN</td>
<td>0.9006666666666666</td>
</tr>
<tr>
<td>Naive Bayes</td>
<td>0.884</td>
</tr>
</table>

<font color=blue>Now no negative values in Experieince in all 5000 entries (4934+66)</font>

<h2><font color="blue">Itertion 2</font>

In [ ]:
#splitting training and testing data from the corrected Dataframe bank_df_corrected
X_train, X_test = train_test_split(bank_df_corrected , test_size=0.3, random_state=1)
y_train = X_train.pop("Personal Loan")
y_test = X_test.pop("Personal Loan")

<font color="blue"><b>Logistic Regression using statsmodel api learn</b></font>

In [ ]:
modelcrorecard2={}

pred_y_df,lgmetric = do_statmodelLR(X_train,X_test,y_train,y_test)
con_matrix = confusion_matrix(pred_y_df.original,pred_y_df.predicted)


In [ ]:
acc1= calculate_accuracy(con_matrix)
modelcrorecard2["Logistic Regression-statmodel"]=acc1
print(acc1)

<font color="blue"><b>Logistic Regression using scikit learn</b></font>

In [ ]:
LRScore, LRpred_labels = do_LR(X_train,y_train,X_test,y_test)
print("Score scikit learn Logistic Regression : "+str(LRScore))
con_matrix2 = confusion_matrix(y_test,LRpred_labels)
acc2 = calculate_accuracy(con_matrix2)
print(acc2)

In [ ]:
modelcrorecard2["Logistic Regression-scikit learn"]=acc2

<font color="blue"><b>KNN</b></font>

In [ ]:
for i in range(1,20):
    KNNscore,KNNpredict = do_KNN(X_train,y_train,y_test,i)# applying K values from 1 to 20
    print("When K value is "+str(i)+" score = "+str(KNNscore))

In [ ]:
optimum_kvalue2 = 3 #selection an ODD value
KNNscore,KNNpredict = do_KNN(X_train,y_train,y_test,optimum_kvalue2)
print("When K value is "+str(optimum_kvalue2)+" score = "+str(KNNscore))
con_matrix3 = confusion_matrix(y_test,KNNpredict)
acc3 = calculate_accuracy(con_matrix3)
modelcrorecard2["KNN"] = acc3
print(acc3)

<font color="blue"><b>Naive Bayes</b></font>

In [ ]:
NBscore, NBpredicted_labels = do_naive_bayes(X_train,y_train,X_test,y_test)
print("Score in Naive Bayes : ",NBscore)
con_matrix4 = confusion_matrix(y_test,NBpredicted_labels)
acc4 = calculate_accuracy(con_matrix4)
modelcrorecard2["Naive Bayes"]=acc4
print(acc4)

<font color="blue">Printing all confusion matrix</font>

In [ ]:
draw_conmmat(y_test,pred_y_df.predicted,"Logistic Regression using StatModel")
draw_conmmat(y_test,LRpred_labels,"Logistic Regression using scikit learn")
draw_conmmat(y_test,KNNpredict,"KNN")
draw_conmmat(y_test,NBpredicted_labels,"NBpredicted_labels")

<font color="blue">Comparing Model scores between two iterations</font>

In [ ]:
for key, val in modelcrorecard2.items():
    print(key,"",val)

<font color='red'>-----------------------<br>
Iteration 1 Results:<br>
-----------------------<br>
LRstatmodel  0.9486666666666667<br>
LRscikitlearn  0.9073333333333333<br>
KNN  0.9026666666666666<br>
Naive Bayes  0.884<br></font><font color=blue>
----------------------<br>
Iteration 2 Results:<br>
----------------------<br>
Logistic Regression-statmodel  0.9466666666666667<br>
Logistic Regression-scikit learn  0.9113333333333333<br>
KNN  0.9026666666666666<br>
Naive Bayes  0.884<br></font>

<table>
<tr>
    <td><b>Model</b></td>
    <td><b>Score<br>Iteration 1</b></td>
    <td><b>Score<br>Iteration 2</b></td>
</tr>
<tr>
<td>Logistic Regression-statmodel</td>
    <td>94.86</td>
<td>94.66</td>
</tr>
<tr>
<td>Logistic Regression-scikit learn</td>
    <td>90.73</td>
<td>91.13</td>
</tr>
<tr>
<td>KNN</td>
    <td>90.27</td>
<td>90.27</td>
</tr>
<tr>
<td>Naive Bayes</td>
    <td>88.4</td>
<td>88.4</td>
</tr>
</table>

<p style="background-color:black;"><font color="yellow">Inference & Conclusion</font></p>

<p>Comparing above results among models, given the problem to compare the results among three classification models(Logistic, K-NN and Naïve Bayes), <font color="red">Logistic Regression</font> statsmodel - seems to give better results for the given data set.</p>
<p>Point here to note there is slight difference between StatsModel API and scikit learn Logistic Regrission for given dataset statsmodel api works / preforms better </p>
<p>
    Logistic Regression perform better as it <font color="red">does not expect the dependent variable to be normally distrubuted,</font> in the give dataset the dependent variable "Personal Loan" is not normally distributed.</p><p>
    Naive Bayes expects all attributes to be independent, here there are few highly dependent attributes were there like Income and Experience, Age and Experience, etc
    </p>
    <p>KNN is nonparametric where Logistic Regression is parametric, with limited attribute set Logistic Regression outperforms. However KNN is also equally good while comparing with LogisticRegression scikit learn package
</p>
    
<p><font color="#07bdf5">More observations</font>
    Attribute <b>Experience </b>has <font color="red">52</font> negative values, probably data entry issue - after replacing them with mean there is a slight increase in score of Logistic Regression using scikit learn from 90.73 to 91.13</p>
<p>
    <ul>
        <li><font color="purple">Take away for Business/Bank:</font> Families with income more than USD100,000 are availed the personal loan during previous campaign, Families Income below $100,000 are less likely to avail personal loan </li>
        </ul>